In [6]:
import tensorflow as tf

print("TensorFlow Version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow Version: 2.14.0
Num GPUs Available:  1


In [9]:
import tensorflow as tf
from keras.layers import SimpleRNN
from keras.models import Sequential
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

model_rnn = Sequential()
model_rnn.add(SimpleRNN(units=20, activation=tf.keras.activations.relu, input_shape=(10,5)))

print(model_rnn.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 20)                520       
                                                                 
Total params: 520 (2.03 KB)
Trainable params: 520 (2.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
